In [1]:
# import dependencies

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from pandas import DataFrame

In [6]:
html = "https://www.zip-codes.com/county/ca-san-diego.asp"
contents = requests.get(html)

soup = BeautifulSoup(contents.content, 'lxml')
# Need to parse through DOM to know which table to scrape
table = soup.find_all('table')[2]
lists = pd.read_html(str(table))

website_data_df = lists[0]
print(f'Initial size of dataframe: {website_data_df.shape}')
website_data_df.head()

Initial size of dataframe: (182, 6)


,0,1,2,3,4,5
0,ZIP Code,Classification,City,Population,Timezone,Area Code(s)
1,ZIP Code 91901,General,Alpine,17403,Pacific,619
2,ZIP Code 91902,General,Bonita,17653,Pacific,619
3,ZIP Code 91903,P.O. Box,Alpine,0,Pacific,619
4,ZIP Code 91905,General,Boulevard,1700,Pacific,619/760


In [9]:
no_population = website_data_df[website_data_df[3] == '0'].index
website_data_df.drop(no_population, inplace=True)
print(f'Updated size of dataframe: {website_data_df.shape}')
website_data_df.head()

Updated size of dataframe: (107, 6)


,0,1,2,3,4,5
0,ZIP Code,Classification,City,Population,Timezone,Area Code(s)
1,ZIP Code 91901,General,Alpine,17403,Pacific,619
2,ZIP Code 91902,General,Bonita,17653,Pacific,619
4,ZIP Code 91905,General,Boulevard,1700,Pacific,619/760
5,ZIP Code 91906,General,Campo,3627,Pacific,619
